## Description of the method
### Assumptions
*Unconfoundedness*\:  
　　Treatment status within the final groups of firms (partitions) trees calculate is random.
$$
\left[
W_{i} \perp\!\!\!\perp \left(Y_{i}(0), Y_{i}(1)\right) \mid X_{i}
\right]
$$

### Notation
*Average Treatment Effect (ATE)*\:
$$ATE=\mathbb{E}[\beta]=\mathbb{E}[Y_1]-\mathbb{E}[Y_0] $$
$$Y_i=Y_{0i}+(Y_{1i}-Y_{0i})W_i $$

\begin{eqnarray*} \underset{\mbox{ Observed Differences in Earnings}}{\mathbb{E}[Y_{i}|W=1]-\mathbb{E}[Y_i|W=0]}&=&\underset{\mbox{ Average treatment effect on the treated}}{\mathbb{E}[Y_{1i}-Y_{0i}|W=1]}\\ &-&\underset{\mbox{ Selection Bias}}{\mathbb{E}[Y_{0i}|W=1]-\mathbb{E}[Y_{0i}|W=0]}\end{eqnarray*}

*Conditional Average Treatment Effect (CATE)*\:  
　　The conditional average treatment effect function is:  
$$\tau(x)=\mathbb{E}\left[Y_{i}(1)-Y_{i}(0) \mid X_{i}=x\right]$$


## Research Motivation
 
　　I generate the data referring to the essay, and its DGP comes from the essay:   
- Huseyin Guleny, Candace E. Jensz, T. Beau Pagex, An application of causal forest in corporate finance: How does financing affect investment? ,2020.04.23.  
 
　　Theoretical references:  
- Athey, S., Imbens, G., 2016. Recursive partitioning for heterogeneous causal effects. Proceedings of the National Academy of Sciences 113, 7353-7360.  
- GuidoWImbens & Donald B Rubin. Causal Inference in Statistics, Social, and Biomedical Sciences. Cambridge University Press, 2015.  
- Stefan Wager & Susan Athey. Estimation and Inference of Heterogeneous Treatment Effects using Random Forests. Journal of the American Statistical Association, 0162-1459.  

## My Project
　　This project aims to compare how well OLS and causal forest work in different situations. Here I will change several sample sizes and models to show the difference.  
　　This project compares the linear and non-linear specification estimations respectively, along three dimensions: bias, precision, and coverage. I measure bias as the difference between the estimated ATE and the true value of ATE of each model, with confidence interval under 97.5% confidence level. Precision is the root mean squared error (RMSE), measured as the standard deviation of the difference between the estimate and the true value. Coverage is the fraction of trials in which a t-test of the estimate compared to the true value rejects at the 5% level.  
　　I visualize bias by line/point/box plots, and present precision (RMSE) and coverage in tables. All else equal, we prefer an estimator with zero bias, low RMSE, and test coverage of 5%.

In [2]:
rm(list = ls())
options(warn =-1)
if (!require("grf")) install.packages("grf")
if (!require("DiagrammeR")) install.packages("DiagrammeR")
if (!require("akima")) install.packages("akima")
if (!require("plotly")) install.packages("plotly")
library(MASS)
library(ggplot2)
library(dplyr)
library(purrr)
library(cowplot)
set.seed(100)

### Model 1: linear model   
$$y = 0.05x_1 - 0.005x_2 + 0.01x_3 + 0.02D + \varepsilon$$

　　$y$ is the dependent variable, the three $x$ variables are additional covariates, $D$ is a
binary variable equal to one if the forcing variable $d$ is greater than zero. I simulate the $x$ and $d$ variables using a multivariate normal distribution. For the vector
$(x1, x2, x3, d)$, I set the mean to (1.40, 5.50, 0.10, 0.20), standard deviation to (1.00, 1.50, 0.30, 0.30),
and use the correlation matrix:
\begin{bmatrix}
1.00&-0.05&-0.30& 0.15 \\
-0.05& 1.00& 0.20& 0.35 \\
-0.30& 0.20& 1.00& 0.50 \\
0.15& 0.35& 0.50& 1.00
\end{bmatrix}
The $\varepsilon$ term has a mean of zero and a standard deviation
of 0.065.

In [7]:
### model 1 ###
# y = 0.05x1 - 0.005x2 + 0.01x3 + 0.02D + eps 
dgp_model1 <- function(N){
  # mean, standard deviation, correlation matrix for generate (x1,x2,x3,d) using mvrnorm
  mu <- c(1.40, 5.50, 0.10, 0.20)
  sd <- c(1.00, 1.50, 0.30, 0.30)
  cor_matrix <- matrix(c( 1.00,-0.05,-0.30, 0.15,
                         -0.05, 1.00, 0.20, 0.35,
                         -0.30, 0.20, 1.00, 0.50,
                          0.15, 0.35, 0.50, 1.00), nrow = 4, ncol = 4)
  beta <- c(0.05, -0.005, 0.01, 0.02) # for x1,x2,x3,D
  # gerate covariation matrix using standard error and correlation matrix
  cov_matrix <- diag(sd) %*% cor_matrix %*% diag(sd)
  # generate x1,x2,x3,d
  mvnorm_data <- mvrnorm(n = N, mu = mu, Sigma = cov_matrix)
  # generate X : (x1, x2, x3)
  poly_max <- 1
  X <- reduce( c(1,1:poly_max), 
               function(X, poly)
                 cbind(X, mvnorm_data[,1]^poly, mvnorm_data[,2]^poly, mvnorm_data[,3]^poly)
  ) %>% subset(select = -c(1))
  if(poly_max == 1) {
    colnames(X) <- c("x1","x2","x3")
  } else {
    colnames(X) <- rep(c("x1","x2","x3"), poly_max - 1) %>%
      paste0(c(rep("", 3),rep(paste0("^", 2:poly_max), each = 3)))
  }
  
  D <- as.integer(mvnorm_data[,4] > 0)
  X <- cbind(X, D)
  eps <- rnorm(N, 0, 0.065)
  y <- X %*% beta + eps
  X <- subset(X, select = c(x1, x2, x3, D))
  
  beta.true <- 0.02
  return(list(X=X, y=y, d=mvnorm_data[,4], beta.true=beta.true))
}

　　To get ATE, bias, RMSE, confidence interval and coverage from ols and causal forest, I write this two function. $X$ is a data frame with x1,x2,x3 and D generated from DGP. $y$ is true y. $beta.true$ is a fixed number for every certain DGP.

In [8]:
###########################
#### Method Comparison ####
###########################

ols<-function(X, y, beta.true){
  X <- cbind(1, X)
  lm <- lm(data = data.frame(X, y), "y ~ x1 + x2 + x3 + D")
  su <- summary(lm)
  beta.hat <- su$coefficients[,"Estimate"]
  y.hat <- lm$fitted.values
  se <- su$coefficients[,"Std. Error"]
  RMSE <- sqrt(sum(su$residuals ^ 2) / length(y))
  if(nrow(su$coefficients) < 5){
    print("coefficients is empty")
  }
  coverage <- su$coefficients["D", 4]
  ATE <- beta.hat["D"]
  bias <- abs(ATE - beta.true) / beta.true * 100
  
  #95% CI of coefficient of D
  upper <- beta.hat["D"] + qnorm(0.975) * se["D"]
  lower <- beta.hat["D"] - qnorm(0.975) * se["D"]
  
  return(list(ATE = ATE, bias = bias, CI = c(lower, upper), RMSE = RMSE, coverage = coverage))
}

forest<-function(X, y, beta.true){
  Y <- y
  W <- X[, "D"]
  X <- subset(X, select = c(x1, x2, x3))
  
  Y.forest <- regression_forest(X, Y)
  Y.hat <- predict(Y.forest)$predictions
  W.forest <- regression_forest(X, W)
  W.hat <- predict(W.forest)$predictions
  
  cf <- causal_forest(X, Y, W, Y.hat = Y.hat, W.hat = W.hat)
  tau.hat <- predict(cf)$predictions
  ATE.forest <- average_treatment_effect(cf)
  tree <- get_tree(cf, 1)
  
  #95% CI of ATE
  upper <- ATE.forest[1] + qnorm(0.975) * ATE.forest[2]
  lower <- ATE.forest[1] - qnorm(0.975) * ATE.forest[2]
  
  RMSE <- sqrt(sum((y - Y.hat) ^ 2) / length(y))
  
  ATE <- ATE.forest[1]
  bias <- abs(ATE - beta.true) / beta.true * 100
  
  test <- test_calibration(cf)
  coverage <- test[1,4]
  return(list(ATE = ATE, bias = bias, CI = c(lower, upper), RMSE = RMSE, coverage = coverage, tree=tree, tau.hat=tau.hat))
}

　　Simulation function is used for quickly generating plot data, by inputting a vector of sample size and a number for iteration times for every data size.  

In [9]:
###########################
####    Simulation     ####
###########################
# Quickly generate plot data.
# Args:
#   sim.times: Number for iteration times for every data size.
#   data.size: Vector of sample size for simulation.
#   dgp_func: Function to generate X, y, beta.true, such as "dgp_model1"
# Returns:
#   Data frame contains ols and causal forest result for every data size and simulation time.
simulation <- function(sim.times, data.size, dgp_func){
  plot.data<-data.frame(ATE = rep(NA, sim.times * length(data.size) * 2), 
                        RMSE = NA,
                        CI_lower = NA,
                        CI_upper = NA,
                        bias = NA,
                        coverage = NA,
                        method = factor(x=rep(c("ols", "forest"), sim.times * length(data.size)), levels = c("ols", "forest")), 
                        sim.time = rep(1:sim.times, each = 2, times = length(data.size)),
                        N = rep(data.size, each = 2 * sim.times))
  index <- 0
  for(j in 1:length(data.size)){
    N = data.size[j]
    for(i in 1:sim.times){
      sim.data <- dgp_func(N)
      
      index <- index + 1
      result.ols<-ols(sim.data$X, sim.data$y, sim.data$beta.true)
      plot.data$ATE[index] <- result.ols$ATE
      plot.data$RMSE[index] <- result.ols$RMSE
      plot.data$CI_lower[index] <- result.ols$CI[1]
      plot.data$CI_upper[index] <- result.ols$CI[2]
      plot.data$bias[index] <- result.ols$bias
      plot.data$coverage[index] <- result.ols$coverage
      
      index <- index + 1
      result.cf<- forest(sim.data$X, sim.data$y, sim.data$beta.true)
      plot.data$ATE[index] <- result.cf$ATE
      plot.data$RMSE[index] <- result.cf$RMSE
      plot.data$CI_lower[index] <- result.cf$CI[1]
      plot.data$CI_upper[index] <- result.cf$CI[2]
      plot.data$bias[index] <- result.cf$bias
      plot.data$coverage[index] <- result.cf$coverage
    }
  }
  return(plot.data)
}

**Simulation 1: modify sample size from 500 to 15000.**  
\* *I use loops in R, but here for convenience I change loops into normal codes to show the intermediate results.*  
\* *Simulation 1 takes hours to run. Change data.size1 for quick look.*


In [12]:
data.size1 <- seq(500, 15000, 100)
sim.data.list1 <- list()

In [13]:
dgp_func <- dgp_model1
name <- "model 1"
i <- 1

# generate plot data
sim.data <- simulation(sim.times = 1, data.size = data.size1, dgp_func)
# sim.data <- sim.data.list1[[i]] # for replay
sim.data.list1[[i]] <- sim.data
beta.true <- dgp_func(2)$beta.true

# bias line plot
plot_bias <- ggplot(sim.data, aes(N)) +
    geom_line(aes(y = bias, color = method)) +
    ylim(min(sim.data$bias), max(sim.data$bias)) +
    labs(title = paste0(name, " bias"))
print(plot_bias)

# ATE line plot
plot_ATE <- ggplot(sim.data, aes(N)) +
    geom_line(aes(y = ATE, color = method), cex = 1) +
    geom_ribbon(aes(ymin = CI_lower, ymax = CI_upper, fill = method), alpha = 0.1) +
    geom_hline(yintercept = beta.true, color = I("black")) +
    ylim(min(sim.data$CI_lower), max(sim.data$CI_upper)) +
    labs(title = paste0(name, " ATE"))
print(plot_ATE)


**Simulation 2: use (500, 1000, 5000, 10000) for sample size and iterate 1000 for every sample size**  
\* *Simulation 2 takes hours to run. Change sim.times for quick look.*

In [ ]:
data.size2 <- c(500, 1000, 5000, 10000)
sim.data.list2 <- list()
mean_sim2 <- data.frame()

In [ ]:
dgp_func <- dgp_model1
name <- "model 1"
i <- 1

sim.data <- simulation(sim.times = 1000, data.size = data.size2, dgp_func)
# sim.data <- sim.data.list2[[i]] # for replay
sim.data.list2[[i]] <- sim.data
beta.true <- dgp_func(2)$beta.true

# generate point and box plot for every data size
ATE_point_plot.list <- list()
ATE_box_plot.list <- list()
for(j in 1:length(data.size2)){
    N <- data.size2[j]
    plot.heigth <- max(beta.true - min(sim.data$ATE), max(sim.data$ATE) - beta.true)
    ATE_point_plot.list[[j]] <- ggplot(sim.data[sim.data$N==N,], aes(sim.time)) +
      geom_point(aes(y = ATE, color = method, shape = method)) +
      geom_hline(yintercept = beta.true, color = I("black")) +
      ylim(beta.true - plot.heigth, beta.true + plot.heigth) +
      labs(subtitle = paste0("sample size = ", N), x = "iteration", y = "ATE")

    ATE_box_plot.list[[j]] <- ggplot(sim.data[sim.data$N==N,], aes(method)) +
      geom_boxplot(aes(y = ATE, color = method)) +
      ylim(min(sim.data$ATE), max(sim.data$ATE)) +
      labs(subtitle = paste0("sample size = ", N), x = NULL)
}
title <- ggdraw() + 
    draw_label(name, fontface = 'bold', x = 0, hjust = 0) +
    theme(plot.margin = margin(0, 0, 0, 20))
ATE_point_plot <- plot_grid(plotlist = ATE_point_plot.list)
ATE_box_plot <- plot_grid(plotlist = ATE_box_plot.list)
# ATE_point.png
print(plot_grid(title, ATE_point_plot, ncol = 1, rel_heights = c(0.1, 1)))
# ATE_box.png
print(plot_grid(title, ATE_box_plot, ncol = 1, rel_heights = c(0.1, 1)))

# generate mean of simulation result data
sim.data_grouped <- group_by(sim.data, method, N)
mean_by_method_N <- summarize(sim.data_grouped, ATE = mean(ATE,na.rm=TRUE), RMSE = mean(RMSE,na.rm=TRUE), bias = mean(bias,na.rm=TRUE),
                                   coverage = mean(coverage,na.rm=TRUE)) %>% mutate(model = name)
mean_sim2 <- rbind(mean_sim2, mean_by_method_N)
print(mean_by_method_N)

　　Simulations for the linear base case demonstrate the following: we can see from plots and table that OLS is the best estimator for sample size of 500 to 5000, since causal forest has higher variance and performs more unstable than OLS.  
　　As sample size increases, MSE decreases for both ols and causal forest, and causal forest is the dominant estimator in large sample size.  
　　Although causal forest and ols have comparable mse for all sample sizes, causal forest deceases its bias as sample size increases and it has lower bias than ols at sample size of 10000. This is because causal forest is data hungry and thus needs large sample size to work well.  

　　To better understand the tree method, I draw a tree plot to show how causal forest works in this process with a sample size of 1000. It shows the sample size as well as the average value of W (ATE) in each branch.

In [ ]:
# plot tree
dgp_func <- dgp_model1
name <- "model 1"
sim.data <- dgp_func(1000)
result.cf<- forest(sim.data$X, sim.data$y, sim.data$beta.true)
plot(result.cf$tree)

### Model 2: polynomial specification
$$y=0.05 x_{1}-0.005 x_{2}+0.01 x_{3}+0.025 x_{1}^{2}-0.01 x_{2}^{2}+0.015 x_{3}^{2}+0.02 D+\varepsilon$$

　　In the following models we will have misspecification problem. Therefore, the treatment effect is biased for OLS model. However, causal forest works pretty well in heterogenous designed model and its ATE value is closer to the true value. We can see from the plots that ATE in OLS (green line) is always waving above the true value, while causal forest (red line) has a fluctuation up and down the true value line.

In [ ]:
### model 2 ###
# y = 0.05x1 + 0.005x2 + 0.01x3 + 0.025x1^2 + 0.01x2^2 + 0.015x3^2 + 0.02D + eps 
dgp_model2 <- function(N){
  # mean, standard deviation, correlation matrix for generate (x1,x2,x3,d) using mvrnorm
  mu <- c(1.40, 5.50, 0.10, 0.20)
  sd <- c(1.00, 1.50, 0.30, 0.30)
  cor_matrix <- matrix(c( 1.00,-0.05,-0.30, 0.15,
                         -0.05, 1.00, 0.20, 0.35,
                         -0.30, 0.20, 1.00, 0.50,
                          0.15, 0.35, 0.50, 1.00), nrow = 4, ncol = 4)
  beta <- c(0.05, -0.005, 0.01, 0.025, -0.01, 0.015, 0.02) # for x1,x2,x3,x1^2,x2^2,x3^2,D
  # gerate covariation matrix using standard error and correlation matrix
  cov_matrix <- diag(sd) %*% cor_matrix %*% diag(sd)
  # generate x1,x2,x3,d
  mvnorm_data <- mvrnorm(n = N, mu = mu, Sigma = cov_matrix)
  # generate X with polynomial: (x1, x2, x3, x1^2, x2^2, x3^2)
  poly_max <- 2
  X <- reduce( c(1,1:poly_max), 
               function(X, poly)
                 cbind(X, mvnorm_data[,1]^poly, mvnorm_data[,2]^poly, mvnorm_data[,3]^poly)
  ) %>% subset(select = -c(1))
  if(poly_max == 1) {
    colnames(X) <- c("x1","x2","x3")
  } else {
    colnames(X) <- rep(c("x1","x2","x3"), poly_max - 1) %>%
      paste0(c(rep("", 3),rep(paste0("^", 2:poly_max), each = 3)))
  }
  
  D <- as.integer(mvnorm_data[,4] > 0)
  X <- cbind(X, D)
  eps <- rnorm(N, 0, 0.065)
  y <- X %*% beta + eps
  X <- subset(X, select = c(x1, x2, x3, D))
  
  beta.true <- 0.02
  return(list(X=X, y=y, d=mvnorm_data[,4], beta.true=beta.true))
}

**Simulation 1**

In [ ]:
dgp_func <- dgp_model2
name <- "model 2"
i <- 2

# generate plot data
sim.data <- simulation(sim.times = 1, data.size = data.size1, dgp_func)
# sim.data <- sim.data.list1[[i]] # for replay
sim.data.list1[[i]] <- sim.data
beta.true <- dgp_func(2)$beta.true

# bias line plot
plot_bias <- ggplot(sim.data, aes(N)) +
    geom_line(aes(y = bias, color = method)) +
    ylim(min(sim.data$bias), max(sim.data$bias)) +
    labs(title = paste0(name, " bias"))
print(plot_bias)

# ATE line plot
plot_ATE <- ggplot(sim.data, aes(N)) +
    geom_line(aes(y = ATE, color = method), cex = 1) +
    geom_ribbon(aes(ymin = CI_lower, ymax = CI_upper, fill = method), alpha = 0.1) +
    geom_hline(yintercept = beta.true, color = I("black")) +
    ylim(min(sim.data$CI_lower), max(sim.data$CI_upper)) +
    labs(title = paste0(name, " ATE"))
print(plot_ATE)

　　As sample size increases, causal forest works better than small sample size case and both OLS and causal forest decrease their variance (improve precision) in this process.

**Simulation 2**

In [ ]:
dgp_func <- dgp_model2
name <- "model 2"
i <- 2

sim.data <- simulation(sim.times = 1000, data.size = data.size2, dgp_func)
# sim.data <- sim.data.list2[[i]] # for replay
sim.data.list2[[i]] <- sim.data
beta.true <- dgp_func(2)$beta.true

# generate point and box plot for every data size
ATE_point_plot.list <- list()
ATE_box_plot.list <- list()
for(j in 1:length(data.size2)){
    N <- data.size2[j]
    plot.heigth <- max(beta.true - min(sim.data$ATE), max(sim.data$ATE) - beta.true)
    ATE_point_plot.list[[j]] <- ggplot(sim.data[sim.data$N==N,], aes(sim.time)) +
      geom_point(aes(y = ATE, color = method, shape = method)) +
      geom_hline(yintercept = beta.true, color = I("black")) +
      ylim(beta.true - plot.heigth, beta.true + plot.heigth) +
      labs(subtitle = paste0("sample size = ", N), x = "iteration", y = "ATE")

    ATE_box_plot.list[[j]] <- ggplot(sim.data[sim.data$N==N,], aes(method)) +
      geom_boxplot(aes(y = ATE, color = method)) +
      ylim(min(sim.data$ATE), max(sim.data$ATE)) +
      labs(subtitle = paste0("sample size = ", N), x = NULL)
}
title <- ggdraw() + 
    draw_label(name, fontface = 'bold', x = 0, hjust = 0) +
    theme(plot.margin = margin(0, 0, 0, 20))
ATE_point_plot <- plot_grid(plotlist = ATE_point_plot.list)
ATE_box_plot <- plot_grid(plotlist = ATE_box_plot.list)
# ATE_point.png
print(plot_grid(title, ATE_point_plot, ncol = 1, rel_heights = c(0.1, 1)))
# ATE_box.png
print(plot_grid(title, ATE_box_plot, ncol = 1, rel_heights = c(0.1, 1)))

# generate mean of simulation result data
sim.data_grouped <- group_by(sim.data, method, N)
mean_by_method_N <- summarize(sim.data_grouped, ATE = mean(ATE,na.rm=TRUE), RMSE = mean(RMSE,na.rm=TRUE), bias = mean(bias,na.rm=TRUE),
                                   coverage = mean(coverage,na.rm=TRUE)) %>% mutate(model = name)
mean_sim2 <- rbind(mean_sim2, mean_by_method_N)
print(mean_by_method_N)

　　Bias is higher for causal forest in small sample size, then it performs better as sample size increases. The reason is listed above that causal forest needs large data to perform well. As sample size increases, both OLS and causal forest work better in variance and coverage.

**plot tree**

In [ ]:
# plot tree
dgp_func <- dgp_model2
name <- "model 2"
sim.data <- dgp_func(1000)
result.cf<- forest(sim.data$X, sim.data$y, sim.data$beta.true)
plot(result.cf$tree)

### Model 3: with interaction terms 
$$y=0.05 x_{1}-0.005 x_{2}+0.01 x_{3} + 0.02D + 0.01x_{1}D + 0.001x_2D + 0.002x_1x_2D + \varepsilon$$
　　In model 3 we have omitted variable problem. Because of the heterogeneous design, causal forest should perform better than OLS. As can be seen from results, OLS has high bias in every specification and, as sample size increases, worse coverage.

In [ ]:
### model 3 ###
# y = 0.05x1 + 0.005x2 + 0.01x3 + 0.02D + tau1*x1*D + tau2*x2*D + tau3*x1*x2*D + eps
dgp_model3 <- function(N){
  # mean, standard deviation, correlation matrix for generate (x1,x2,x3,d) using mvrnorm
  mu <- c(1.40, 5.50, 0.10, 0.20)
  sd <- c(1.00, 1.50, 0.30, 0.30)
  cor_matrix <- matrix(c( 1.00,-0.05,-0.30, 0.15,
                         -0.05, 1.00, 0.20, 0.35,
                         -0.30, 0.20, 1.00, 0.50,
                          0.15, 0.35, 0.50, 1.00), nrow = 4, ncol = 4)
  beta <- c(0.05, -0.005, 0.01, 0.02) # for x1,x2,x3,D
  tau <- c(0.01, 0.001, 0.002)
  # gerate covariation matrix using standard error and correlation matrix
  cov_matrix <- diag(sd) %*% cor_matrix %*% diag(sd)
  # generate x1,x2,x3,d
  mvnorm_data <- mvrnorm(n = N, mu = mu, Sigma = cov_matrix)
  # generate X with polynomial: (x1, x2, x3)
  poly_max <- 1
  X <- reduce( c(1,1:poly_max), 
               function(X, poly)
                 cbind(X, mvnorm_data[,1]^poly, mvnorm_data[,2]^poly, mvnorm_data[,3]^poly)
  ) %>% subset(select = -c(1))
  if(poly_max == 1) {
    colnames(X) <- c("x1","x2","x3")
  } else {
    colnames(X) <- rep(c("x1","x2","x3"), poly_max - 1) %>%
      paste0(c(rep("", 3),rep(paste0("^", 2:poly_max), each = 3)))
  }
  
  D <- as.integer(mvnorm_data[,4] > 0)
  X <- cbind(X, D)
  eps <- rnorm(N, 0, 0.065)
  y <- X %*% beta + eps
  
  # add intersection
  X_D <- cbind(x1_D = X[,"x1"] * D, x2_D = X[,"x2"] * D, x1_x2_D = X[,"x1"]*X[,"x2"]*D)
  y <- y + X_D %*% tau
  
  X <- subset(X, select = c(x1, x2, x3, D))
  
  beta.true <- 0.02 + tau %*% c(mu[1:2], (mu[1]*mu[2] + cov_matrix[1,2])) # E(X1)*E(X2)+Cov(X1,X2)=E(X1X2)
  return(list(X=X, y=y, d=mvnorm_data[,4], beta.true=beta.true))
}

**Simulation 1**

In [ ]:
dgp_func <- dgp_model3
name <- "model 3"
i <- 3

# generate plot data
sim.data <- simulation(sim.times = 1, data.size = data.size1, dgp_func)
# sim.data <- sim.data.list1[[i]] # for replay
sim.data.list1[[i]] <- sim.data
beta.true <- dgp_func(2)$beta.true

# bias line plot
plot_bias <- ggplot(sim.data, aes(N)) +
    geom_line(aes(y = bias, color = method)) +
    ylim(min(sim.data$bias), max(sim.data$bias)) +
    labs(title = paste0(name, " bias"))
print(plot_bias)

# ATE line plot
plot_ATE <- ggplot(sim.data, aes(N)) +
    geom_line(aes(y = ATE, color = method), cex = 1) +
    geom_ribbon(aes(ymin = CI_lower, ymax = CI_upper, fill = method), alpha = 0.1) +
    geom_hline(yintercept = beta.true, color = I("black")) +
    ylim(min(sim.data$CI_lower), max(sim.data$CI_upper)) +
    labs(title = paste0(name, " ATE"))
print(plot_ATE)

**Simulation 2**

In [ ]:
dgp_func <- dgp_model3
name <- "model 3"
i <- 3

sim.data <- simulation(sim.times = 1000, data.size = data.size2, dgp_func)
# sim.data <- sim.data.list2[[i]] # for replay
sim.data.list2[[i]] <- sim.data
beta.true <- dgp_func(2)$beta.true

# generate point and box plot for every data size
ATE_point_plot.list <- list()
ATE_box_plot.list <- list()
for(j in 1:length(data.size2)){
    N <- data.size2[j]
    plot.heigth <- max(beta.true - min(sim.data$ATE), max(sim.data$ATE) - beta.true)
    ATE_point_plot.list[[j]] <- ggplot(sim.data[sim.data$N==N,], aes(sim.time)) +
      geom_point(aes(y = ATE, color = method, shape = method)) +
      geom_hline(yintercept = beta.true, color = I("black")) +
      ylim(beta.true - plot.heigth, beta.true + plot.heigth) +
      labs(subtitle = paste0("sample size = ", N), x = "iteration", y = "ATE")

    ATE_box_plot.list[[j]] <- ggplot(sim.data[sim.data$N==N,], aes(method)) +
      geom_boxplot(aes(y = ATE, color = method)) +
      ylim(min(sim.data$ATE), max(sim.data$ATE)) +
      labs(subtitle = paste0("sample size = ", N), x = NULL)
}
title <- ggdraw() + 
    draw_label(name, fontface = 'bold', x = 0, hjust = 0) +
    theme(plot.margin = margin(0, 0, 0, 20))
ATE_point_plot <- plot_grid(plotlist = ATE_point_plot.list)
ATE_box_plot <- plot_grid(plotlist = ATE_box_plot.list)
# ATE_point.png
print(plot_grid(title, ATE_point_plot, ncol = 1, rel_heights = c(0.1, 1)))
# ATE_box.png
print(plot_grid(title, ATE_box_plot, ncol = 1, rel_heights = c(0.1, 1)))

# generate mean of simulation result data
sim.data_grouped <- group_by(sim.data, method, N)
mean_by_method_N <- summarize(sim.data_grouped, ATE = mean(ATE,na.rm=TRUE), RMSE = mean(RMSE,na.rm=TRUE), bias = mean(bias,na.rm=TRUE),
                                   coverage = mean(coverage,na.rm=TRUE)) %>% mutate(model = name)
mean_sim2 <- rbind(mean_sim2, mean_by_method_N)
print(mean_by_method_N)

**plot tree**

In [ ]:
# plot tree
dgp_func <- dgp_model3
name <- "model 3"
sim.data <- dgp_func(1000)
result.cf<- forest(sim.data$X, sim.data$y, sim.data$beta.true)
plot(result.cf$tree)

### *comparison of the tree plots*
　　We can see from the tree plots that tree branches increase from model 1 to 3, and meanwhile bias decreases. From left to right are plots for Model 1 to model 3 respectively, we can see that bias of causal forest becomes smaller in this process. (True ATE is the red line, estimated ATE is the blue line).

In [ ]:
sim_model <- list(dgp_model1, dgp_model2, dgp_model3)
model_names <- c("model 1", "model 2", "model 3")

for(i in c(1:length(sim_model))){
  dgp_func <- sim_model[[i]]
  name <- model_names[i]
  sim.data <- dgp_func(1000)
  result.cf<- forest(sim.data$X, sim.data$y, sim.data$beta.true)
  # Evaluate the estimate using a histogram
  print(ggplot(data.frame(effect=result.cf$tau.hat, y=sim.data$y, treated=sim.data$X[,"D"]), aes(x=effect)) +
    geom_histogram(bins=60) +
    geom_vline(xintercept=result.cf$ATE, linetype="dotted", color="blue", size=1.5) +
    geom_vline(xintercept=sim.data$beta.true, color = "red", size=1.5) +
    labs(title = paste0(name, " tau.hat histogram")))
}

　　Next step we will calculate CATE value for x1 ad x2, and present it in rainbow plot and 3D plot. I equally divide the simulation into 50 partitions with sample size of 500000, but there are missing data due to the Gaussian distribution of X.

　　Use function "CATE" to caculate CATE for specify x1 and x2 groups. Since x1 and x2 is continuous, we can't get CATE from value of x1 and x2. I divide the simulation into 50 partitions, and use the mean of every partition to stand for x1 and x2 value.

In [ ]:
CATE <- function(dgp.data, x1_groups, x2_groups, model_name){
  
  x1_group_levels <- sort(unique(x1_groups))
  x1_value_levels <- map_dbl(x1_group_levels, function(x1_group) mean(dgp.data$X[x1_groups==x1_group,"x1"])) # value equals mean of x1 in its group
  x2_group_levels <- sort(unique(x2_groups))
  x2_value_levels <- map_dbl(x2_group_levels, function(x2_group) mean(dgp.data$X[x2_groups==x2_group,"x2"]))
  
  CATE.data <- data.frame(ATE = rep(NA, length(x1_group_levels) * length(x2_group_levels) * 2),
                        method = factor(x=rep(c("ols", "forest"), length(x1_group_levels) * length(x2_group_levels)), levels = c("ols", "forest")),
                        x2_group = rep(x2_group_levels, each = 2, times = length(x1_group_levels)),
                        x1_group = rep(x1_group_levels, each = 2 * length(x2_group_levels)),
                        x2_value = rep(x2_value_levels, each = 2, times = length(x1_value_levels)),
                        x1_value = rep(x1_value_levels, each = 2 * length(x2_value_levels)))
  CATE.matrix.forest <- matrix(NA, nrow = length(x1_value_levels), ncol = length(x2_value_levels), dimnames = list(x1_value_levels,x2_value_levels))
  CATE.matrix.ols <- matrix(NA, nrow = length(x1_value_levels), ncol = length(x2_value_levels), dimnames = list(x1_value_levels,x2_value_levels))
  CATE.matrix.beta_true <- matrix(NA, nrow = length(x1_value_levels), ncol = length(x2_value_levels), dimnames = list(x1_value_levels,x2_value_levels))
  index <- 1
  for (i in 1:length(x1_group_levels)) {
    for(j in 1:length(x2_group_levels)) {
      x1_group <- x1_group_levels[i]
      x2_group <- x2_group_levels[j]
      
      sub.X <- dgp.data$X[(x1_groups == x1_group & x2_groups == x2_group),]
      sub.y <- dgp.data$y[(x1_groups == x1_group & x2_groups == x2_group)]
      
      if(length(sub.y) > 4){
        if(sum(sub.X[,"D"]) != 0 && sum(sub.X[,"D"]) != nrow(sub.X)){
          result.ols <- ols(sub.X, sub.y, dgp.data$beta.true)
          result.cf <- forest(sub.X, sub.y, dgp.data$beta.true)
          
          CATE.data$ATE[index] <- result.ols$ATE
          CATE.data$ATE[index+1] <- result.cf$ATE
          
          CATE.matrix.ols[i,j] <- result.ols$ATE
          CATE.matrix.forest[i,j] <- result.cf$ATE
          
          if(model_name == "model 3"){
            CATE.matrix.beta_true[i,j] <- 0.02 + c(0.01, 0.001, 0.002) %*% c(mean(sub.X[,"x1"]), mean(sub.X[,"x2"]), mean(sub.X[,"x1"]*sub.X[,"x2"]))
          } else if (model_name == "model 4"){
            CATE.matrix.beta_true[i,j] <- 0.02 + c(0.1, 0.1, -0.02, -0.01, 0.01) %*% 
              c(mean(sub.X[,"x1"]), mean(sub.X[,"x2"]), mean(sub.X[,"x1"]^2), mean(sub.X[,"x2"]^2), mean(sub.X[,"x1"]*sub.X[,"x2"]))
          }
        }
      }
      index <- index + 2
    }
  }
  return(list(CATE.data = CATE.data, CATE.matrix.forest = CATE.matrix.forest, CATE.matrix.ols = CATE.matrix.ols, CATE.matrix.beta_true = CATE.matrix.beta_true, x1_value_levels = x1_value_levels, x2_value_levels = x2_value_levels))
}

In [ ]:
dgp_func <- dgp_model3
name <- "model 3"

dgp.data <- dgp_func(500000)
CATE.result <- CATE(dgp.data, ntile(dgp.data$X[,"x1"],50), ntile(dgp.data$X[,"x2"],50), name)

# rainbow plot
print(ggplot(subset(CATE.result$CATE.data, method=="forest"), aes(x=x1_group, y=x2_group, color = ATE)) +
      geom_point(size = 4) +
      scale_color_gradientn(colours = rainbow(10)) +
      labs(title = paste0(name, " CATE rainbow of forest")))

# box plot
box.plot.data <- CATE.result$CATE.data %>% subset(ATE < 2 & method=="forest") %>% mutate(x1_value = as.factor(.$x1_value))
print(ggplot(box.plot.data, aes(x1_value)) +
      geom_boxplot(aes(y = ATE)) +
      labs(title = paste0(name, " CATE boxplot of forest")))

# Gridded Bivariate Interpolation plot by package "akima"
im <- with(CATE.result$CATE.data %>% subset(method == "forest" & !is.na(ATE)), interp(x1_value,x2_value,ATE))
with(im,image(x,y,z, xlab = "x1 value", ylab = "x2 value"))

# 3D plot by package "plotly"
# plotly_forest
plot_ly(x=CATE.result$x1_value_levels,y=CATE.result$x2_value_levels,z=CATE.result$CATE.matrix.forest, type="surface")
# plotly_ols
plot_ly(x=CATE.result$x1_value_levels,y=CATE.result$x2_value_levels,z=CATE.result$CATE.matrix.ols, type="surface")
# plotly_true
plot_ly(x=CATE.result$x1_value_levels,y=CATE.result$x2_value_levels,z=CATE.result$CATE.matrix.beta_true, type="surface")


### Model 4: with polynomials and interaction terms
$$y=0.05 x_1-0.005 x_2+0.01 x_3 + 0.02D + 0.01x_1D + 0.001x_2D -0.02x_1^2D-0.01x_2^2D+0.01x_1x_2D+ \varepsilon$$

In [ ]:
### model 4 ###
# y = 0.05x1 + 0.005x2 + 0.01x3 + 0.02D + tau1*x1*D + tau2*x2*D + tau3*x1^2*D + tau4*x2^2*D + tau5*x1*x2*D + eps
dgp_model4 <- function(N){
  # mean, standard deviation, correlation matrix for generate (x1,x2,x3,d) using mvrnorm
  mu <- c(1.40, 5.50, 0.10, 0.20)
  sd <- c(1.00, 1.50, 0.30, 0.30)
  cor_matrix <- matrix(c( 1.00,-0.05,-0.30, 0.15,
                         -0.05, 1.00, 0.20, 0.35,
                         -0.30, 0.20, 1.00, 0.50,
                          0.15, 0.35, 0.50, 1.00), nrow = 4, ncol = 4)
  beta <- c(0.05, -0.005, 0.01, 0.02) # for x1,x2,x3,D
  tau <- c(0.1, 0.1, -0.02, -0.01, 0.01)
  # gerate covariation matrix using standard error and correlation matrix
  cov_matrix <- diag(sd) %*% cor_matrix %*% diag(sd)
  # generate x1,x2,x3,d
  mvnorm_data <- mvrnorm(n = N, mu = mu, Sigma = cov_matrix)
  # generate X: (x1, x2, x3)
  poly_max <- 1
  X <- reduce( c(1,1:poly_max), 
               function(X, poly)
                 cbind(X, mvnorm_data[,1]^poly, mvnorm_data[,2]^poly, mvnorm_data[,3]^poly)
  ) %>% subset(select = -c(1))
  if(poly_max == 1) {
    colnames(X) <- c("x1","x2","x3")
  } else {
    colnames(X) <- rep(c("x1","x2","x3"), poly_max - 1) %>%
      paste0(c(rep("", 3),rep(paste0("^", 2:poly_max), each = 3)))
  }
  
  D <- as.integer(mvnorm_data[,4] > 0)
  X <- cbind(X, D)
  eps <- rnorm(N, 0, 0.065)
  y <- X %*% beta + eps
  
  # add intersection
  X_D <- cbind(x1_D = X[,"x1"] * D, x2_D = X[,"x2"] * D, x12_D = X[,"x1"]^2 * D, x22_D = X[,"x2"]^2 * D, x1_x2_D = X[,"x1"] * X[,"x2"] * D)
  y <- y + X_D %*% tau
  
  X <- subset(X, select = c(x1, x2, x3, D))
  
  beta.true <- 0.02 + tau %*% c(mu[1:2], (mu[1:2]^2 + sd[1:2]^2), (mu[1]*mu[2] + cov_matrix[1,2]))
  return(list(X=X, y=y, d=mvnorm_data[,4], beta.true=beta.true))
}

**Simulation 2**

In [ ]:
dgp_func <- dgp_model4
name <- "model 4"
i <- 4

sim.data <- simulation(sim.times = 1000, data.size = data.size2, dgp_func)
# sim.data <- sim.data.list2[[i]] # for replay
sim.data.list2[[i]] <- sim.data
beta.true <- dgp_func(2)$beta.true

# generate point and box plot for every data size
ATE_point_plot.list <- list()
ATE_box_plot.list <- list()
for(j in 1:length(data.size2)){
    N <- data.size2[j]
    plot.heigth <- max(beta.true - min(sim.data$ATE), max(sim.data$ATE) - beta.true)
    ATE_point_plot.list[[j]] <- ggplot(sim.data[sim.data$N==N,], aes(sim.time)) +
      geom_point(aes(y = ATE, color = method, shape = method)) +
      geom_hline(yintercept = beta.true, color = I("black")) +
      ylim(beta.true - plot.heigth, beta.true + plot.heigth) +
      labs(subtitle = paste0("sample size = ", N), x = "iteration", y = "ATE")

    ATE_box_plot.list[[j]] <- ggplot(sim.data[sim.data$N==N,], aes(method)) +
      geom_boxplot(aes(y = ATE, color = method)) +
      ylim(min(sim.data$ATE), max(sim.data$ATE)) +
      labs(subtitle = paste0("sample size = ", N), x = NULL)
}
title <- ggdraw() + 
    draw_label(name, fontface = 'bold', x = 0, hjust = 0) +
    theme(plot.margin = margin(0, 0, 0, 20))
ATE_point_plot <- plot_grid(plotlist = ATE_point_plot.list)
ATE_box_plot <- plot_grid(plotlist = ATE_box_plot.list)
# ATE_point.png
print(plot_grid(title, ATE_point_plot, ncol = 1, rel_heights = c(0.1, 1)))
# ATE_box.png
print(plot_grid(title, ATE_box_plot, ncol = 1, rel_heights = c(0.1, 1)))

# generate mean of simulation result data
sim.data_grouped <- group_by(sim.data, method, N)
mean_by_method_N <- summarize(sim.data_grouped, ATE = mean(ATE,na.rm=TRUE), RMSE = mean(RMSE,na.rm=TRUE), bias = mean(bias,na.rm=TRUE),
                                   coverage = mean(coverage,na.rm=TRUE)) %>% mutate(model = name)
mean_sim2 <- rbind(mean_sim2, mean_by_method_N)
print(mean_by_method_N)

In [ ]:
dgp_func <- dgp_model4
name <- "model 4"

dgp.data <- dgp_func(500000)
CATE.result <- CATE(dgp.data, ntile(dgp.data$X[,"x1"],50), ntile(dgp.data$X[,"x2"],50), name)

# rainbow plot
print(ggplot(subset(CATE.result$CATE.data, method=="forest"), aes(x=x1_group, y=x2_group, color = ATE)) +
      geom_point(size = 4) +
      scale_color_gradientn(colours = rainbow(10)) +
      labs(title = paste0(name, " CATE rainbow of forest")))

# box plot
box.plot.data <- CATE.result$CATE.data %>% subset(ATE < 2 & method=="forest") %>% mutate(x1_value = as.factor(.$x1_value))
print(ggplot(box.plot.data, aes(x1_value)) +
      geom_boxplot(aes(y = ATE)) +
      labs(title = paste0(name, " CATE boxplot of forest")))

# Gridded Bivariate Interpolation plot by package "akima"
im <- with(CATE.result$CATE.data %>% subset(method == "forest" & !is.na(ATE)), interp(x1_value,x2_value,ATE))
with(im,image(x,y,z, xlab = "x1 value", ylab = "x2 value"))

# 3D plot by package "plotly"
# plotly_forest
plot_ly(x=CATE.result$x1_value_levels,y=CATE.result$x2_value_levels,z=CATE.result$CATE.matrix.forest, type="surface")
# plotly_ols
plot_ly(x=CATE.result$x1_value_levels,y=CATE.result$x2_value_levels,z=CATE.result$CATE.matrix.ols, type="surface")
# plotly_true
plot_ly(x=CATE.result$x1_value_levels,y=CATE.result$x2_value_levels,z=CATE.result$CATE.matrix.beta_true, type="surface")


Finally, by the aggerated box plot for 4 models, we can have a clear look about it.

In [ ]:
# box plot all in one for sim2
select.model <- c("model 1", "model 2", "model 3", "model 4")
select.data.size <- c(1000, 5000, 10000)
model.box_plot.list <- list()
for(model_name in select.model){
  sim.data <- sim.data.list2[[which(model_name == model_names)]]
  sim.data <- sim.data[sim.data$N %in% select.data.size,]
  model.box_plot.list[[I(model_name)]] <- ggplot(sim.data, aes(method)) +
    geom_boxplot(aes(y = ATE, color = method)) +
    ylim(min(sim.data$ATE), max(sim.data$ATE)) +
    facet_wrap(~N, nrow = 1) +
    labs(title = model_name, x = NULL)
}
print(plot_grid(plotlist = model.box_plot.list, align = "hv"))

## Empirical Application
　　As we can see from the project results, causal forest treatment has great extendibility and it has lots of empirical applications. For example, the essay I refer to is about an application of causal forest in corporate finance. Here I list two reasons for its goodness as an estimating tool:  
　　First, many questions in economics or finance involve evaluating the treatment effect of a binary status or decision, in which case we need to deal with treatment effect heterogeneity. This is the type of question causal forest is designed to address.  
　　Second, plenty of macroeconomic studies tend to have large data to deal with, thus providing a sufficient sample size. Our simulations show the importance of sample size in a causal forest estimation to recovering unbiased treatment effects, and confirm sample sizes as small as 5,000 to 15,000 are sufficient for a causal forest estimation with low error.  
